In [1]:
!git clone https://github.com/imoutoforder/deep_text_2.git

Cloning into 'deep_text_2'...
remote: Enumerating objects: 302341, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 302341 (delta 12), reused 21 (delta 7), pack-reused 302306
Receiving objects: 100% (302341/302341), 425.46 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Checking out files: 100% (280243/280243), done.


In [8]:
# %rm -rd deep_text_2/EasyOCR/trainer/all_data/fa_val/
# %rm -rd deep_text_2/EasyOCR/trainer/all_data/
# %rm -rd deep_text_2/.git/
# !zip -r deep.zip ./deep_text_2/
!cp ./deep.zip drive/MyDrive/ANN/deep.zip

In [ ]:
#@title Default title text
!pip install python-bidi
!pip install arabic_reshaper



In [ ]:
%cd deep_text_2/EasyOCR/trainer/

/content/deep_text_2/EasyOCR/trainer


In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [ ]:
%ls ./all_data/fa_val/ | wc -l


20001


In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r') as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    print(opt)
    print(opt['train_data'])
    print(opt['select_data'])
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'train_data','labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt



In [ ]:
opt = get_config("config_files/fa_filtered_config.yaml")
opt.character 
print(len(opt.character))

{'number': '۱۲۳۴۵۶۷۸۹۰', 'symbol': '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €', 'lang_char': '0123456789!"#$%&\\\'()*+,-./:;<=>?@[\\\\]^_`{|}~ abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ٠١٢٣٤٥٦٧٨٩«»؟،؛ءآأؤإئااًبةتثجحخدذرزسشصضطظعغفقكلمنهوىيًٌٍَُِّْٰٓٔٱٹپچڈڑژکڭگںھۀہۂۃۆۇۈۋیېےۓە', 'experiment_name': 'fa_filtered', 'train_data': 'all_data', 'valid_data': 'all_data/fa_val', 'manualSeed': 1111, 'workers': 4, 'batch_size': 32, 'num_iter': 25000, 'valInterval': 5000, 'saved_model': '', 'FT': False, 'optim': False, 'lr': 1.0, 'beta1': 0.9, 'rho': 0.95, 'eps': 1e-08, 'grad_clip': 5, 'select_data': 'fa_train_filtered', 'batch_ratio': '1', 'total_data_usage_ratio': 1.0, 'batch_max_length': 34, 'imgH': 45, 'imgW': 120, 'rgb': False, 'contrast_adjust': 0.0, 'sensitive': True, 'PAD': True, 'data_filtering_off': False, 'Transformation': 'None', 'FeatureExtraction': 'ResNet', 'SequenceModeling': 'BiLSTM', 'Prediction': 'CTC', 'num_fiducial': 20, 'input_channel': 1, 'output_channel': 512, 'hidden_siz

In [ ]:
!apt-get install imagemagick
!mogrify -resize 120x64! './all_data/fa_train_filtered/train_data/*.jpg'

In [ ]:
!mogrify -resize 120x64! './all_data/fa_val/*.jpg'

In [ ]:
train(opt, amp=False)

In [ ]:
from utils import CTCLabelConverter
conv = CTCLabelConverter(opt.character)

In [ ]:
%cp deep_text_2/EasyOCR/trainer/config_files/fa_filtered_config_old_2.yaml deep_text_2/EasyOCR/trainer/config_files/fa_filtered_config.yaml

In [ ]:
#%cd ./../../../
!git clone https://github.com/imoutoforder/TextGenerator.git

Cloning into 'TextGenerator'...
remote: Enumerating objects: 633, done.
remote: Counting objects: 100% (633/633), done.
remote: Compressing objects: 100% (567/567), done.
remote: Total 633 (delta 65), reused 631 (delta 63), pack-reused 0
Receiving objects: 100% (633/633), 66.62 MiB | 29.66 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
%ls
%cd ./TextGenerator/

drive/  sample_data/  TextGenerator/
/content/TextGenerator


In [ ]:
import bidi
import arabic_reshaper
!python run.py -l fa -c 200000 -i fa_eval.txt -na 2 -t 4 -bl 1 -rbl --output_dir ./fa_evall


100% 200000/200000 [11:28<00:00, 290.66it/s]


In [ ]:
!zip -r val.zip ./fa_evall

In [ ]:
!cp ./val.zip ./../drive/MyDrive/ANN/val.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -r deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train_data/
!rm -r deep_text_2/EasyOCR/trainer/all_data/fa_val/


In [ ]:
!cp drive/MyDrive/ANN/train.zip deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/
!cp drive/MyDrive/ANN/val.zip deep_text_2/EasyOCR/trainer/all_data/

In [ ]:
!unzip deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train.zip -d deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train_data/
!unzip deep_text_2/EasyOCR/trainer/all_data/val.zip -d deep_text_2/EasyOCR/trainer/all_data/fa_val/


In [ ]:
!rm deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train.zip
!rm deep_text_2/EasyOCR/trainer/all_data/val.zip

In [ ]:
# Modifying the labels
# !mv -v deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train_data/fa_train/* deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train_data/
import os
import shutil

root = "deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train_data/"
source = os.path.join(root, "fa_train")
for it in os.listdir(source):
  print(it)
  shutil.move(os.path.join(source, it), os.path.join(root, it))
  

!rm -d deep_text_2/EasyOCR/trainer/all_data/fa_train_filtered/train_data/fa_train/

In [ ]:
#%cd deep_text_2/
#!git init
# !touch .gitignore1
# !echo EasyOCR/trainer/all_data/ > .gitignore
# !git add --all
# !git status -s
!git config --local user.email "my.dmrc@gmail.com"
# !git commit -m "dataless"

/bin/bash: open: command not found
